Analysis:

1. Which country has the most movies?
2. Comparision of total TV shows vs Movies count?
3. Who are the most active director?
4. What are the trend of movies counts for past 10 years?
5. Which types of movie ratings are more popular?

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
root = '/kaggle/input/netflix-shows/'
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df = pd.read_csv(
    os.path.join(root, 'netflix_titles.csv'))
df

In [ ]:
sns.heatmap(df.isnull(), cmap = 'inferno')

In [ ]:
df.dropna(subset=['date_added'],axis = 0, inplace = True)
df[df['rating'].isnull()]
replace_rating = {67: 'TV-14', 2359: 'TV-14', 3660: 'PG-13', 3736: 'TV-14', 3737: 'TV-14', 3738: 'TV-14', 4323: 'TV-14'}
for i, rating in replace_rating.items():
    df.loc[i, 'rating'] = rating

In [ ]:
df.nunique()/df.shape[0]*100

In [ ]:
#Adding values to missing elements for country
df['country'] = df['country'].fillna('United States')
df['fist_country_in_agg'] = df['country'].apply(lambda x: x.split(",")[0]) #when there are multiple countries in a cell

#Adding an column for season_count
df['season_count'] = df['duration'].apply(lambda x : x.split(" ")[0] if "Season" in x else "")
df['season_count']


In [ ]:
#  Top10 Countries Based on the Movie Count
from collections import Counter

import matplotlib.pyplot as plt
country_movies = df[df.type=="Movie"].country.value_counts()

#Creating a series for a multiple countries 
multiple_labels = country_movies[country_movies.index.str.contains(",")]
multiple_labels = multiple_labels.index.str.split(", ")

#Creating an array separating group values into a single row
a=[]
for i in range(len(multiple_labels)):
    for j in range(len(multiple_labels[i])):
        a.append(multiple_labels[i][j])

a = country_movies.append(pd.Series(Counter(a)))
b = a.groupby(by= a.index).sum()
country_movies_df = b[~b.index.str.contains(",")]

country_movies_df = country_movies_df.sort_values(ascending = False)[0:10]
plt.figure(figsize=(12, 3))
top10country = country_movies_df.sort_values(ascending = False)[0:10]
sns.barplot(x = top10country.index, y= top10country.values)
plt.show()


In [ ]:
movie = df[df['type']== 'Movie']['type'].count()
TV = df[df['type']== 'TV Show']['type'].count()
plt.bar(['Movie', 'TV'], height=[movie, TV], color=['red','green'], visible = True)
plt.title('Comparision between TV and Movie shows count')
plt.xlabel('Medium')
plt.ylabel('Count')
plt.show()

In [ ]:
df_movie = df[df.type=='Movie']
df_movie_graph = df_movie.groupby('director', as_index= False).count()[['director','show_id']].sort_values(by='show_id', ascending=False)[:8]
plt.figure(figsize=(15,10))
plt.bar(df_movie_graph['director'], df_movie_graph['show_id'], color=['blue'], visible = True)
plt.title('Most movies by director')
plt.xlabel('Director')
plt.ylabel('Count')
plt.show()

In [ ]:
df_movie_graph = df_movie.groupby('release_year', as_index= False).count()[['release_year','show_id']].sort_values(by='release_year', ascending=False)[:10]
plt.bar(df_movie_graph['release_year'], df_movie_graph['show_id'], color=['blue'], visible = True)
plt.title('Movies count for last 10 years')
plt.xlabel('Year')
plt.ylabel('Movie')
plt.show()

In [ ]:
df_movie_graph = df_movie.groupby('rating', as_index= False).count()[['rating','show_id']].sort_values(by='show_id', ascending=False)[:10]
plt.bar(df_movie_graph['rating'], df_movie_graph['show_id'], color=['blue'], visible = True)
plt.title('Movies with top rating count')
plt.xlabel('Rating')
plt.ylabel('Movie')
plt.show()